<a href="https://colab.research.google.com/github/fanat503/IronGPT/blob/main/gpt_Tolstoy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F



!wget -O modern_text.txt https://www.gutenberg.org/cache/epub/2600/pg2600.txt

with open('modern_text.txt', 'r', encoding='utf-8') as f:
    text = f.read()


chars = sorted(list(set(text)))
vocab_size = len(chars)

stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 64
block_size = 256

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

--2026-02-26 17:20:09--  https://www.gutenberg.org/cache/epub/2600/pg2600.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3359652 (3.2M) [text/plain]
Saving to: ‘modern_text.txt’

modern_text.txt     100%[===================>]   3.20M  7.26MB/s    in 0.4s    

2026-02-26 17:20:10 (7.26 MB/s) - ‘modern_text.txt’ saved [3359652/3359652]



In [9]:

batch_size = 64
block_size = 256
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'

n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2


class Head(nn.Module):
    """ one head of self-attention """
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size,bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        pass

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)   # (B, T, head_size)
        q = self.query(x) # (B, T, head_size)
        v = self.value(x) # (B, T, head_size)

        wei = q @ k.transpose(-2,-1) * (k.shape[-1] ** -0.5)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        probs = F.softmax(wei, dim=-1)
        self.drop = nn.Dropout(dropout)
        out = probs @ v
        return out



class MultiHeadAttention(nn.Module):
  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(n_embd,n_embd)

  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)
    self.proj(out)
    self.drop = nn.Dropout(dropout)
    return out

class FeedForward(nn.Module):
  def __init__(self, n_embd):
    super().__init__()

    self.net = nn.Sequential(
        nn.Linear(n_embd, 4 * n_embd),
        nn.ReLU(),
        nn.Linear(n_embd*4, n_embd),
    )

  def forward(self,x):
    return self.net(x)

class Block(nn.Module):
  def __init__(self, n_embd, n_head):
    super().__init__()
    head_size = n_embd // n_head
    self.sa = MultiHeadAttention(n_head, head_size)
    self.ffw = FeedForward(n_embd)
    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)

  def forward(self,x):
    x = x + self.sa(self.ln1(x))
    x = x + self.sa(self.ln2(x))

    return x

class BigramLanguageModel(nn.Module):
  def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.drop = nn.Dropout(dropout)


  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -block_size:]

        logits, loss = self(idx_cond)

        logits = logits[:, -1, :]


        probs = F.softmax(logits, dim=-1) # (B, C)

        idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)

        idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

    return idx



  def forward(self, idx, targets=None):
    B,T = idx.shape
    tok_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device))
    x = tok_emb + pos_emb
    x = self.drop(x)
    x = self.blocks(x)
    self.ln_f(x)
    logits = self.lm_head(x)
    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

if __name__ == '__main__':
    model = BigramLanguageModel()

    dummy_idx = torch.randint(0, vocab_size, (4, 8)) # B=4, T=8

    logits, loss = model(dummy_idx)


In [ ]:
model.to(device)


optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

max_iters = 5000

for iter in range(max_iters):

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


    if iter % 300 == 0:
        print(f"Iteration {iter}: Loss = {loss.item():.4f}")



In [11]:
def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -block_size:]

        logits, loss = self(idx_cond)

        logits = logits[:, -1, :]

        probs = F.softmax(logits, dim=-1)

        idx_next = torch.multinomial(probs, num_samples=1)

        idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

    return idx

In [ ]:

model.eval()

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_idx = model.generate(context, max_new_tokens=3000)
print(decode(generated_idx[0].tolist()))

model.train()